In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import pypdf
from pinecone import Pinecone, ServerlessSpec


In [2]:
PINECONE_API_KEY = <insert pinecone api>

In [3]:
#slits the data chunks
loader = PyPDFDirectoryLoader("<>") # name of the local folder to read the pdf
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

7020


In [4]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "test" #name of the index in pinecone


In [5]:
#
import time

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,          # change this to the length of each vector embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
# using the sentence-transformers/all-MiniLM-L6-v2 from HuggingFace for the embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [7]:
# Pinecone Vector Store
vector_store = PineconeVectorStore(index=index, embedding=embedding)

In [8]:
# add the chunks to the pinecone DB

from uuid import uuid4
from langchain_core.documents import Document

uids = [str(uuid4()) for _ in range(len(text_chunks))]
vector_store.add_documents(documents=text_chunks, ids=uuids)

In [ ]:
# A sample similarity search
query=<> # sample query
results = vector_store.similarity_search(
    query,
    k=4
)
print("Result", results)

In [11]:
# form the LLM model
llm = CTransformers(model="<>", # insert the model which you want to use
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.1})

In [12]:
# Prompt template
prompt_template="""
Give am answer based on the text provided. Try to be exact. Don't guess the answer.

Conext : {context}
Question: {question}

Return the most relevant answer
"""

In [13]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "questions"])
chain_type_kwargs={"prompt": prompt}

In [14]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [15]:
from builtins import input

while True:
    user_input = input("Enter the query :")
    query_result = qa({"query": user_input})
    print("Response : ", query_result["result"])